In [1]:
from matplotlib import pyplot as plt
from scipy.special import softmax
from itertools import product
import numpy as np
##
from utils import *
from model import *
from analysis import *
##
import time
import seaborn as sns
from glob import glob as glob
sns.set_context('talk')

%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
gsname = 'gs0209'

### load data

In [3]:
gsdf = pd.read_csv('data/%s-summdf.csv'%gsname).astype(float)

In [4]:
## load human data
hdf = pd.read_csv('../human_data.csv')
humanD = {}
for c in ['blocked','interleaved','early','middle','late']:
  humanD[c[0]] = hdf.loc[:,('%s mean')%c].values.T[-40:].mean()
## test acc: difference to human
for c in ['b','i','e','m','l']:
  gsdf['%s-test_diff'%c] = np.abs(gsdf.loc[:,'testacc-%s'%c] - humanD[c])
## block+interleaved mse
gsdf.loc[:,'mse-bi'] = gsdf.loc[:,'mse-i']+gsdf.loc[:,'mse-b']

### plt single condition

In [5]:
def save_and_close(param_str,rank,mse):
  plt.title(param_str[:120])
  plt.savefig("figures/top40/%s-%i-mse%.3f-%s.png"%(gsname,rank,mse,param_str))
  plt.close('all')
  
def pipeline(row,ns=50):
  """ 
  each step of the pipeline involves 
    running a csw experiment on a param contained in row
    calculating a metric on exp_batch_data
    plotting and saving
  """
  mse = dict(row)['mse-bi'].round(3)
  ## RUN EXP
  args,paramstr = get_argsD(row)
  exp_batch_data = run_batch_exp_curr(ns,args,condL) # [curr,seeds,{data}]
  ## ACCURACY
  plt_acc(exp_batch_data)
  save_and_close("acc-%s"%paramstr,rank,mse)
  ## LC COUNTS VIOLIN
  plt_LC_violins(exp_batch_data)
  save_and_close("lc-%s"%paramstr,rank,mse)
  ## ajdusted rand
  plt_arscores(exp_batch_data)
  save_and_close("ar-%s"%paramstr,rank,mse)

# top 40 MSE analyses

In [6]:
## select top40 MSE
top40gsdf = gsdf.sort_values('mse-bi').iloc[:40]

In [7]:
## LOOP
for rank,(idx,row) in enumerate(top40gsdf.iterrows()):
  print(rank)
  pipeline(row)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
